# Review Classification Pipeline - Google Colab Setup

This notebook sets up and runs the complete review classification pipeline in Google Colab.

**Features:**
- Ollama-style prompt classification (using OpenAI API)
- HuggingFace transformer models
- Ensemble classification
- GPT pseudo-labeling
- Complete evaluation metrics

## 1. Environment Setup

In [ ]:
# Install required packages
!pip install -q transformers torch pandas scikit-learn openai tqdm
!pip install -q datasets accelerate

print("Packages installed successfully!")

## 2. Project Structure Setup

In [ ]:
# Create directory structure
import os
import pandas as pd
import json
from pathlib import Path

# Create all necessary directories
directories = [
    'src/config', 'src/core', 'src/pseudo_labelling', 'src/utils',
    'data/sample', 'data/raw', 'data/processed',
    'results/predictions', 'results/evaluations', 'results/reports',
    'logs/pipeline_logs', 'models/cache', 'prompts'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    # Create __init__.py files for Python packages
    if directory.startswith('src/'):
        with open(f'{directory}/__init__.py', 'w') as f:
            f.write('# Package init\n')

print("Directory structure created!")
print("Created directories:", directories)

## 3. Sample Data Creation

In [ ]:
# Create sample review data
sample_data = {
    'id': [1, 2, 3, 4, 5],
    'text': [
        "Great product, highly recommend!",
        "Terrible service, waste of money", 
        "Check out this amazing deal at example.com",
        "The staff was rude and unprofessional",
        "Overpriced scammers. Society is doomed."
    ],
    'gold_label': ['APPROVE', 'REJECT', 'REJECT', 'REJECT', 'REJECT'],
    'gold_category': ['None', 'Irrelevant', 'No_Ads', 'Irrelevant', 'Rant_No_Visit']
}

df = pd.DataFrame(sample_data)
df.to_csv('data/sample/sample_reviews.csv', index=False)

print("Sample data created!")
print(df)

## 4. Configuration Setup

In [ ]:
# Create configuration classes
config_code = '''
from dataclasses import dataclass, field
import os

@dataclass
class ModelConfig:
    """Configuration for model settings"""
    openai_model: str = "gpt-3.5-turbo"
    hf_sentiment_model: str = "distilbert-base-uncased-finetuned-sst-2-english"
    hf_toxicity_model: str = "unitary/toxic-bert"
    hf_zero_shot_model: str = "facebook/bart-large-mnli"
    
    sentiment_threshold: float = 0.7
    toxicity_threshold: float = 0.5
    zero_shot_threshold: float = 0.7
    ensemble_tau: float = 0.55

@dataclass
class PipelineConfig:
    """Main pipeline configuration"""
    model: ModelConfig = field(default_factory=ModelConfig)
    openai_api_key: str = ""
    max_gpt_calls_per_day: int = 1000
    gpt_cost_limit: float = 10.0
    
config = PipelineConfig()
'''

with open('src/config/pipeline_config.py', 'w') as f:
    f.write(config_code)

print("Configuration created!")

## 5. Constants and Prompts

In [ ]:
# Create constants
constants_code = '''
POLICY_CATEGORIES = {
    'NO_ADS': 'No_Ads',
    'IRRELEVANT': 'Irrelevant', 
    'RANT_NO_VISIT': 'Rant_No_Visit',
    'NONE': 'None'
}

LABELS = {
    'APPROVE': 'APPROVE',
    'REJECT': 'REJECT'
}

GPT_PRICING = {
    "gpt-3.5-turbo": 0.002,
    "gpt-4": 0.03,
    "gpt-4-turbo": 0.01,
    "gpt-4o": 0.005
}

ZERO_SHOT_LABELS = [
    "an advertisement or promotional solicitation",
    "off-topic or unrelated content", 
    "a generic negative rant without evidence",
    "a relevant review of an actual visit"
]
'''

with open('src/core/constants.py', 'w') as f:
    f.write(constants_code)

# Create prompt templates
prompts_code = '''
NO_ADS_SYSTEM = """You are analyzing Google reviews to detect advertising/promotional content.
Classify as REJECT if the review contains promotional solicitations, referral codes, or advertising.
Classify as APPROVE if it appears to be a genuine review experience."""

IRRELEVANT_SYSTEM = """You are analyzing Google reviews to detect off-topic content.
Classify as REJECT if the review is unrelated to the business or contains irrelevant content.
Classify as APPROVE if it discusses the actual business or service."""

RANT_NO_VISIT_SYSTEM = """You are analyzing Google reviews to detect generic negative rants.
Classify as REJECT if the review appears to be a generic complaint without evidence of visiting.
Classify as APPROVE if it shows evidence of an actual experience."""

def build_prompt(system_prompt, review_text, few_shots=None):
    """Build a complete prompt for classification"""
    prompt = f"{system_prompt}\n\nReview: {review_text}\n\nRespond with JSON: {{\"label\": \"APPROVE/REJECT\", \"rationale\": \"reason\", \"confidence\": 0.8}}"
    return prompt
'''

with open('prompts/policy_prompts.py', 'w') as f:
    f.write(prompts_code)

print("Constants and prompts created!")

## 6. API Key Setup

In [ ]:
# Set up API key (replace with your actual key)
import os
from google.colab import userdata

# Option 1: Use Colab secrets (recommended)
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print("✅ OpenAI API key loaded from Colab secrets")
except:
    # Option 2: Manual input (less secure)
    import getpass
    OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")
    print("✅ OpenAI API key entered manually")

# Set environment variable
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print("\n To add API key to Colab secrets:")
print("1. Click the 🔑 key icon in the left sidebar")
print("2. Add new secret: OPENAI_API_KEY")
print("3. Restart runtime and run this cell again")

## 7. HuggingFace Pipeline

In [ ]:
# HuggingFace-based classification
from transformers import pipeline
import pandas as pd
import torch

# Check if GPU is available
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device == 0 else 'CPU'}")

def run_hf_pipeline(df, output_file="results/predictions/predictions_hf.csv"):
    """Run HuggingFace pipeline classification"""
    print("Loading HuggingFace models...")
    
    # Load models
    sentiment_classifier = pipeline(
        "sentiment-analysis", 
        model="distilbert-base-uncased-finetuned-sst-2-english",
        device=device
    )
    
    zero_shot_classifier = pipeline(
        "zero-shot-classification",
        model="facebook/bart-large-mnli", 
        device=device
    )
    
    results = []
    
    for _, row in df.iterrows():
        text = row['text']
        
        # Zero-shot classification
        labels = [
            "advertisement or promotion",
            "off-topic content",
            "negative rant without visit",
            "genuine review"
        ]
        
        zs_result = zero_shot_classifier(text, labels)
        top_label = zs_result['labels'][0]
        confidence = zs_result['scores'][0]
        
        # Determine final classification
        if top_label == "genuine review" and confidence > 0.7:
            pred_label = "APPROVE"
            pred_category = "None"
        else:
            pred_label = "REJECT"
            if "advertisement" in top_label:
                pred_category = "No_Ads"
            elif "off-topic" in top_label:
                pred_category = "Irrelevant"
            else:
                pred_category = "Rant_No_Visit"
        
        results.append({
            'id': row['id'],
            'text': text,
            'pred_label': pred_label,
            'pred_category': pred_category,
            'confidence': confidence
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"HuggingFace results saved to {output_file}")
    
    return results_df

# Load sample data and run HF pipeline
df = pd.read_csv('data/sample/sample_reviews.csv')
hf_results = run_hf_pipeline(df)
print("\n📊 HuggingFace Results:")
print(hf_results[['id', 'text', 'pred_label', 'pred_category']].to_string(index=False))

## 8. GPT-Based Classification (Pseudo-Labeling)

In [ ]:
# GPT-based classification and pseudo-labeling
import openai
import json
import time
from tqdm import tqdm

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.environ['OPENAI_API_KEY'])

def classify_with_gpt(text, system_prompt):
    """Classify a single review using GPT"""
    prompt = f"{system_prompt}\n\nReview: {text}\n\nRespond with JSON: {{\"label\": \"APPROVE/REJECT\", \"rationale\": \"reason\", \"confidence\": 0.8}}"
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that analyzes reviews."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150,
            temperature=0.1
        )
        
        result_text = response.choices[0].message.content.strip()
        # Try to parse JSON
        try:
            return json.loads(result_text)
        except:
            # Fallback parsing
            if "REJECT" in result_text.upper():
                return {"label": "REJECT", "rationale": "GPT classified as reject", "confidence": 0.8}
            else:
                return {"label": "APPROVE", "rationale": "GPT classified as approve", "confidence": 0.8}
    except Exception as e:
        print(f"Error with GPT API: {e}")
        return {"label": "APPROVE", "rationale": "API error", "confidence": 0.0}

def run_gpt_pipeline(df, output_file="results/predictions/predictions_gpt.csv"):
    """Run GPT-based classification pipeline"""
    print("Running GPT classification...")
    
    # Define policy prompts
    prompts = {
        "No_Ads": """Analyze this review for advertising/promotional content. 
        Classify as REJECT if it contains promotional solicitations, referral codes, or advertising.
        Classify as APPROVE if it appears to be a genuine review.""",
        
        "Irrelevant": """Analyze this review for relevance to the business.
        Classify as REJECT if it's off-topic or unrelated to the business.
        Classify as APPROVE if it discusses the actual business.""",
        
        "Rant_No_Visit": """Analyze this review for evidence of an actual visit.
        Classify as REJECT if it appears to be a generic rant without visiting.
        Classify as APPROVE if it shows evidence of an actual experience."""
    }
    
    results = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="GPT Classification"):
        text = row['text']
        violations = []
        
        # Check each policy
        for policy, prompt in prompts.items():
            result = classify_with_gpt(text, prompt)
            if result['label'] == 'REJECT':
                violations.append((policy, result['confidence']))
            time.sleep(0.1)  # Rate limiting
        
        # Determine final classification
        if violations:
            # Choose violation with highest confidence
            best_violation = max(violations, key=lambda x: x[1])
            pred_label = "REJECT"
            pred_category = best_violation[0]
        else:
            pred_label = "APPROVE"
            pred_category = "None"
        
        results.append({
            'id': row['id'],
            'text': text,
            'pred_label': pred_label,
            'pred_category': pred_category,
            'confidence': 0.8
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)
    print(f"GPT results saved to {output_file}")
    
    return results_df

# Run GPT pipeline
gpt_results = run_gpt_pipeline(df)
print("\nGPT Results:")
print(gpt_results[['id', 'text', 'pred_label', 'pred_category']].to_string(index=False))

## 9. Evaluation and Comparison

In [ ]:
# Evaluation function
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def evaluate_predictions(gold_df, pred_df, pipeline_name):
    """Evaluate predictions against gold standard"""
    merged = gold_df.merge(pred_df[['id', 'pred_label', 'pred_category']], on='id')
    
    print(f"\n {pipeline_name} Evaluation:")
    print("=" * 50)
    
    # Binary classification (APPROVE/REJECT)
    print("\n APPROVE/REJECT Classification:")
    accuracy = accuracy_score(merged['gold_label'], merged['pred_label'])
    print(f"Accuracy: {accuracy:.2%}")
    print("\nClassification Report:")
    print(classification_report(merged['gold_label'], merged['pred_label']))
    
    # Category classification (for REJECT cases)
    reject_cases = merged[merged['gold_label'] == 'REJECT']
    if len(reject_cases) > 0:
        print("\n Category Classification (REJECT cases only):")
        cat_accuracy = accuracy_score(reject_cases['gold_category'], reject_cases['pred_category'])
        print(f"Category Accuracy: {cat_accuracy:.2%}")
        print("\nCategory Report:")
        print(classification_report(reject_cases['gold_category'], reject_cases['pred_category']))
    
    return accuracy

# Load gold standard
gold_df = pd.read_csv('data/sample/sample_reviews.csv')

# Evaluate both pipelines
hf_accuracy = evaluate_predictions(gold_df, hf_results, "HuggingFace")
gpt_accuracy = evaluate_predictions(gold_df, gpt_results, "GPT")

print(f"\n FINAL COMPARISON:")
print(f"HuggingFace Accuracy: {hf_accuracy:.2%}")
print(f"GPT Accuracy: {gpt_accuracy:.2%}")
print(f"Best Pipeline: {'HuggingFace' if hf_accuracy > gpt_accuracy else 'GPT'}")

## 10. Ensemble Pipeline

In [ ]:
# Ensemble classification combining HF and GPT
def run_ensemble_pipeline(df, hf_results, gpt_results, tau=0.55):
    """Combine HuggingFace and GPT predictions"""
    print(f"Running Ensemble with tau={tau}")
    
    ensemble_results = []
    
    for _, row in df.iterrows():
        review_id = row['id']
        
        # Get predictions from both pipelines
        hf_pred = hf_results[hf_results['id'] == review_id].iloc[0]
        gpt_pred = gpt_results[gpt_results['id'] == review_id].iloc[0]
        
        # Agreement check
        if hf_pred['pred_label'] == gpt_pred['pred_label']:
            # Both agree
            final_label = hf_pred['pred_label']
            final_category = hf_pred['pred_category']
            confidence = 0.9  # High confidence when both agree
        else:
            # Disagreement - use HF model with high confidence threshold
            if hf_pred['confidence'] > tau:
                final_label = hf_pred['pred_label']
                final_category = hf_pred['pred_category']
            else:
                final_label = gpt_pred['pred_label']
                final_category = gpt_pred['pred_category']
            confidence = 0.6  # Lower confidence on disagreement
        
        ensemble_results.append({
            'id': review_id,
            'text': row['text'],
            'pred_label': final_label,
            'pred_category': final_category,
            'confidence': confidence,
            'hf_label': hf_pred['pred_label'],
            'gpt_label': gpt_pred['pred_label'],
            'agreement': hf_pred['pred_label'] == gpt_pred['pred_label']
        })
    
    ensemble_df = pd.DataFrame(ensemble_results)
    ensemble_df.to_csv('results/predictions/predictions_ensemble.csv', index=False)
    
    print(f"Ensemble results saved!")
    print(f"Agreement rate: {ensemble_df['agreement'].mean():.2%}")
    
    return ensemble_df

# Run ensemble
ensemble_results = run_ensemble_pipeline(df, hf_results, gpt_results)
ensemble_accuracy = evaluate_predictions(gold_df, ensemble_results, "Ensemble")

print("\nEnsemble Results:")
print(ensemble_results[['id', 'text', 'pred_label', 'agreement']].to_string(index=False))

## 11. 📋 Final Summary

In [ ]:
# Final summary and comparison
print("\nPIPELINE EXECUTION COMPLETE!")
print("=" * 50)

print("\nFINAL RESULTS SUMMARY:")
print(f"HuggingFace Accuracy: {hf_accuracy:.2%}")
print(f"GPT Accuracy: {gpt_accuracy:.2%}")
print(f"Ensemble Accuracy: {ensemble_accuracy:.2%}")

print("\nGenerated Files:")
files = [
    'results/predictions/predictions_hf.csv',
    'results/predictions/predictions_gpt.csv', 
    'results/predictions/predictions_ensemble.csv'
]

for file in files:
    if os.path.exists(file):
        size = os.path.getsize(file)
        print(f"  {file} ({size} bytes)")
    else:
        print(f"  {file} (not found)")

print("\nNext Steps:")
print("  1. Download the CSV files to analyze results")
print("  2. Upload your own review data to data/sample/")
print("  3. Modify thresholds and models for better performance")
print("  4. Use this as a pseudo-labeling system for larger datasets")